In [3]:
import json, hashlib, base64
from mysql.connector import pooling

with open('../../mysql.json') as f:
    config_str = f.read()
config = json.loads(config_str)
pool = pooling.MySQLConnectionPool(pool_name="mypool", pool_size=3, **config)